In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-17 14:28:33--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-11-17 14:28:34 (22.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print('length of dataset in characters', len(text))

length of dataset in characters 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('asta la vista, baby!'))
print(decode(encode('asta la vista, baby!')))

[39, 57, 58, 39, 1, 50, 39, 1, 60, 47, 57, 58, 39, 6, 1, 40, 39, 40, 63, 2]
asta la vista, baby!


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [18]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [19]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is: {target.item()}')

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [23]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('------'*14)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------------------------------------------------------------------------------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the t

In [28]:
print(xb, '\n', yb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
            

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros(1, 1, dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [48]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [56]:
batch_size = 32
for steps in range(10000):
    
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4133121967315674


In [58]:
print(decode(m.generate(torch.zeros(1, 1, dtype=torch.long), max_new_tokens=500)[0].tolist()))


Topad, sucut:
O,
ASt, je I che, t H:
Thosmicor t we aj y in scofire ste,
ARLere s nd mam ooum dropeld d mou we tlll,ieal RKIORDornonche toull ad the whyorupesickers avey he dsks:
theabr mor'd m iceinceyore.
Kibres,

Myorkee bing INalotane IIO:
Nde!
STh, ard hou Gomifserd o.
Ginth w ngour te lingout,

I ofo sarorirt:

Ahon thes the. t itere tiner R lerred; ge y thinourse semea S:
gharomer iou stits.
DUSe m:
BRSTomateehidl e:
AEOKIEinirul Groue!

My O:
gor ser!

To:
Bu har nth brthigro ariusathere


### The mathematical trick in self-attention

In [59]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [60]:
x[0, 1]

tensor([-0.3596, -0.9152])

In [61]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)
        

In [64]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [63]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [69]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

In [71]:
b

tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])

In [70]:
c

tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])

In [75]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [76]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [78]:
xb.shape

torch.Size([32, 8])

##### Token and Positional Embedding

In [80]:
n_embd = 32
tok_emb = nn.Embedding(vocab_size, n_embd)
pos_emb = nn.Embedding(block_size, n_embd)

In [93]:
tok_emb(xb)[0][1]

tensor([ 0.6697, -1.2521,  1.1158, -0.0279, -1.0747, -0.6714,  0.4771, -1.1568,
         0.4973,  0.6844, -1.4450, -1.1042, -1.5878, -0.9785, -1.2732,  0.2967,
         0.6546,  0.1393,  0.3629,  0.6463, -2.2236,  0.9753,  0.4814, -0.8089,
        -0.8841, -1.4150,  0.2092, -1.9765, -0.1158,  2.1447,  0.0951,  0.2028],
       grad_fn=<SelectBackward0>)

In [104]:
pos_emb(torch.tensor([1]))

tensor([[ 8.5244e-01, -3.8668e-01, -3.5624e-01, -1.2950e+00,  6.7834e-01,
         -6.9135e-01, -1.4179e-01,  1.7871e+00,  7.0614e-01, -1.3353e+00,
         -1.5920e+00,  1.0171e+00,  1.8328e-01, -5.4270e-01,  3.3544e-01,
          3.7481e-01,  3.4608e-01,  5.4498e-01,  4.5434e-01,  1.5803e+00,
         -1.6677e-01,  2.0581e-01, -6.8263e-01,  7.4351e-01, -1.1180e+00,
          1.6345e+00, -1.1432e+00, -3.1102e-01, -9.9311e-04, -1.6174e+00,
          1.5543e+00, -9.5562e-01]], grad_fn=<EmbeddingBackward0>)

In [111]:
the_sum = tok_emb(xb) + pos_emb(torch.arange(8))
the_sum[0][1]

tensor([ 1.5222, -1.6388,  0.7595, -1.3229, -0.3964, -1.3627,  0.3353,  0.6303,
         1.2034, -0.6508, -3.0370, -0.0871, -1.4045, -1.5212, -0.9378,  0.6715,
         1.0007,  0.6843,  0.8173,  2.2266, -2.3904,  1.1811, -0.2012, -0.0654,
        -2.0021,  0.2195, -0.9340, -2.2875, -0.1168,  0.5272,  1.6493, -0.7528],
       grad_fn=<SelectBackward0>)

In [114]:
torch.randn((2, 4, 3)).transpose(-1, -2)

tensor([[[ 2.3405,  0.8483, -0.2178, -1.0603],
         [ 0.6097,  0.8767, -0.1887, -0.6252],
         [-1.7136, -0.6807,  1.5465, -0.8588]],

        [[ 0.1199,  1.0055,  0.1814,  0.5579],
         [ 0.4522, -0.7347,  0.3167,  0.5253],
         [-0.0231,  0.6856,  1.0003,  0.1320]]])

In [115]:
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))

v = value(x)
out = wei @ v

In [119]:
nums = torch.arange(24).view(2, 4, 3)
nums

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23]]])

In [123]:
print(nums.transpose(-1, -2).shape)
nums.transpose(-1, -2)

torch.Size([2, 3, 4])


tensor([[[ 0,  3,  6,  9],
         [ 1,  4,  7, 10],
         [ 2,  5,  8, 11]],

        [[12, 15, 18, 21],
         [13, 16, 19, 22],
         [14, 17, 20, 23]]])

In [124]:
print(nums.transpose(-2, -1).shape)
nums.transpose(-2, -1)

torch.Size([2, 3, 4])


tensor([[[ 0,  3,  6,  9],
         [ 1,  4,  7, 10],
         [ 2,  5,  8, 11]],

        [[12, 15, 18, 21],
         [13, 16, 19, 22],
         [14, 17, 20, 23]]])

In [147]:
catx = torch.randn(2, 3, 2)
caty = torch.randn(2, 3, 2)

# print(catx, '\n', caty)
torch.cat([catx, caty], dim=1)

tensor([[[-2.5444e+00, -2.2703e+00],
         [-5.1033e-04, -6.5894e-02],
         [-8.2419e-01, -9.3407e-01],
         [ 2.1025e-01,  2.5601e-01],
         [ 1.1659e+00, -6.2549e-01],
         [-9.4064e-03, -1.1743e+00]],

        [[ 2.1643e-01,  1.4509e+00],
         [ 4.4863e-01,  1.0755e+00],
         [-4.5472e-01,  8.7508e-01],
         [ 1.3388e+00,  4.0735e-01],
         [ 9.0126e-01, -9.8742e-01],
         [-3.3586e-01, -1.0824e-01]]])

In [146]:
catx

tensor([[[-0.0694, -0.1324],
         [-0.7923,  0.1051],
         [-0.8158, -0.0756]],

        [[ 0.2739,  0.9594],
         [ 1.1674, -1.2919],
         [ 0.6870, -1.6423]]])